In [1]:
import json
import requests
from pprint import pprint
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine
import json
import pandas as pd
from config import password

In [2]:
with open('orders.json', 'r') as j:
    orders_data = json.load(j)
    #print(pprint(orders_data))

In [3]:
# Create a function that takes in a date argument and returns the rate for that day
def get_exchange(date):
    # Assign the API url to a variable based on website: 
    # https://exchangeratesapi.io/
    url = f"https://api.exchangeratesapi.io/{date}?symbols=CAD&base=USD"
    # Make request and store response
    response = requests.get(url)
    # Extract the json from the response &return the exchange rate
    return response.json()["rates"]["CAD"]

In [4]:
#Loop through the list of dictionaries and add the currency_rate to each one
#By calling the get_exchange function created earlier
for item in orders_data:
    #use a try & except here in case the API call fails,
    #So it doesn't break the run if we can't get some data.
    try:
        date = item["created_at"].split("T")[0]
        exch_rate = get_exchange(date)
        item["currency_rate"] = exch_rate
        #print(exch_rate)
    except:
        item["currency_rate"] = None

#print(pprint(orders_data))

In [5]:
#write the data into a new file
with open('updated_orders_data.json', 'w') as outfile:
    json.dump(orders_data, outfile)

In [6]:
#create a datfram frame from the data for easy manipulation
complete_df = pd.DataFrame(orders_data)

In [7]:
#create empty dataframes for customer and items
cust_columns = ["id", "name", "email"]
cust_df = pd.DataFrame(columns = cust_columns)

item_columns = ["id", "product_id", "product_sku", "product_name", "price"]
item_df = pd.DataFrame(columns = item_columns)


In [8]:
#rename the columns
# use at the end instead, for better reference
#### rename(columns={customer": "customer_id", "item": "item_id"})

#check for duplicates
#print(len(complete_df[complete_df["id"].duplicated()]))

#Expanding the multiple items inside the lists in the line_item coluumn
orders_df = complete_df
orders_df = orders_df.explode('line_items').reset_index(drop=True)
#orders_df.head()

In [9]:
#Extract all the items from customer
for index, row in orders_df.iterrows():
    #update the item_df & cust_df, will drop duplicates later
    #to improve code you can do a check before this step
    item_df = item_df.append(row["line_items"], ignore_index=True)
    cust_df = cust_df.append(row["customer"], ignore_index=True)
    #now that we've stored the item data and customer data elsewhere
    #we can now leave just the id's for both columns
    orders_df.at[index, "customer"] = row["customer"]["id"]
    orders_df.at[index,"line_items"] = row["line_items"]["id"]
    

In [10]:
## Remove duplicates from the items & customer tables
# sorting by first name 
cust_df.sort_values("id", inplace=True) 
item_df.sort_values("id", inplace=True)
# dropping duplicate values 
cust_df.drop_duplicates(keep="first",inplace=True) 
item_df.drop_duplicates(keep="first",inplace=True) 
#reset index
cust_df.reset_index(drop=True, inplace=True)
item_df.reset_index(drop=True, inplace=True)

In [11]:
orders_df.rename(columns={"customer": "customer_id", 
                          "line_items": "item_id"}, inplace=True)

In [16]:
orders_df.head()

,id,customer_id,total_price,created_at,item_id,currency_rate
0,8369263756632563,47178,79.39,2020-03-07T14:31:11Z,610448,1.342008
1,8369263756632563,47178,79.39,2020-03-07T14:31:11Z,997208,1.342008
2,7262170348080494,94720,86.98,2020-03-01T09:16:30Z,997208,1.344356
3,7262170348080494,94720,86.98,2020-03-01T09:16:30Z,759658,1.344356
4,7722055557038194,59933,36.99,2020-01-01T09:17:03Z,759658,1.299448


In [18]:
#Establish a connection to postgreSQL
connection_string = f"postgres:{password}@localhost:5432/orders_db"
engine = create_engine(f'postgresql://{connection_string}')

In [19]:
# Confirm tables
print(engine.table_names())

['customer', 'orders', 'item']


In [20]:
#name refers to database name
#if_exist = 'append', bbecause the table was already created in PostGres using PgAdmin
item_df.to_sql(name='item', con=engine, if_exists='append', index=False)
cust_df.to_sql(name='customer', con=engine, if_exists='append', index=False)
orders_df.to_sql(name='orders', con=engine, if_exists='append', index=False)


In [21]:
!pip freeze requirements.txt

alabaster==0.7.12
anaconda-client==1.7.2
anaconda-navigator==1.9.12
anaconda-project==0.8.3
applaunchservices==0.2.1
appnope==0.1.0
appscript==1.0.1
argh==0.26.2
asgiref==3.2.10
asn1crypto==1.3.0
astroid==2.3.3
astropy==4.0
atomicwrites==1.3.0
attrs==19.3.0
autopep8==1.4.4
Babel==2.8.0
backcall==0.1.0
backports.functools-lru-cache==1.6.1
backports.shutil-get-terminal-size==1.0.0
backports.tempfile==1.0
backports.weakref==1.0.post1
beautifulsoup4==4.8.2
bitarray==1.2.1
bkcharts==0.2
bleach==3.1.0
bokeh==1.4.0
boto==2.49.0
Bottleneck==1.3.2
certifi==2019.11.28
cffi==1.14.0
chardet==3.0.4
Click==7.0
cloudpickle==1.3.0
clyent==1.2.2
colorama==0.4.3
conda==4.8.2
conda-build==3.18.11
conda-package-handling==1.6.0
conda-verify==3.4.2
contextlib2==0.6.0.post1
cryptography==2.8
cycler==0.10.0
Cython==0.29.15
cytoolz==0.10.1
dask==2.11.0
decorator==4.4.1
defusedxml==0.6.0
demjson==2.2.4
diff-match-patch==20181111
distributed==2.11.0
Django==3.0.9
docutils==0.16
entrypoints==0.3
et-xmlfile==1.0.1